* TF-IDF 주석처리된 코드 입니다.

In [ ]:
import os
from collections import defaultdict
from collections import OrderedDict

class TFIDFCaculator(object):
    def __init__(self, path):
        self.path = path
        self.tf_dict = {}
        self.df_dict = defaultdict(int)

    # 각 문서별 단어별 tf dictionary 생성
    def build_tf_dict(self):
        for root, dirs, files in os.walk(self.path): 
            for f in files: # 각 문서에 대해 순회
                self.tf_dict[f] = defaultdict(int) # 문서별로 이름을 key로 갖는 dict 생성
                with open(os.path.join(self.path, f), 'r') as fout:
                    data = fout.read() # 각 문서의 모든 내용을 읽어들임
                    # 각 내용을 공백으로 잘라서 소문자로 변경 후, 리스트로 만듦
                    words = [word.lower() for word in data.split()] 

                    for w in words:
                        # 앞서 만든 리스트를 순회하면서 각 단어가 출현할 때마 카운팅
                        # f는 문서이름, w는 각 단어를 의미
                        self.tf_dict[f][w] += 1
                        
                        
    # 문서에 존재하는 모든 단어들에 대해 document frequency 생성
    def build_df_dict(self):
        
        # 문서내에 존재하는 모든 단어 추출
        all_keys = list(set([key for doc in self.tf_dict for key in self.tf_dict[doc]]))
        
        for key in all_keys: # 해당 단어 리스트를 순회
            for root, dirs, files in os.walk(self.path):
                for f in files:
                    with open(os.path.join(self.path, f)) as fout:
                        data = fout.read() 
                        words = [word.lower() for word in data.split()]  
                        # 단어들이 전체 문서에서 몇번 출현했는지 빈도 확인
                        if key in words:
                            self.df_dict[key] += 1
         
    # tf, df를 이용하여 tf-idf 결과 생성
    def get_tf_idf_dict(self, reverse = True):
        tf_idf_dict = {}
        
        # doc은 각 문서의 명칭
        for doc in self.tf_dict:
            tf_idf_dict[doc] = {}
            # key는 단어, value는 해당 단어가 doc에서 나타난 빈도수를 의미
            for key, value in self.tf_dict[doc].items():
                # 그것을 df_dict에 저장되어있는 출현한 문서수인 self.df_dict[key]로 나눔
                tf_idf_dict[doc][key] = float(value) / self.df_dict[key]
        
        # 각 문서를 빈도수에 따라 정렬
        for doc in tf_idf_dict:
            tf_idf_dict_by_doc = tf_idf_dict[doc]
            tf_idf_dict_by_doc = OrderedDict(sorted(tf_idf_dict_by_doc.items(), key = lambda x : x[1], reverse = reverse))
            tf_idf_dict[doc] = tf_idf_dict_by_doc
        return tf_idf_dict
            
            
path = os.getcwd() + '/data_set'            
tf_idf = TFIDFCaculator(path)            

tf_idf.build_tf_dict()
tf_idf.build_df_dict()

for doc, tf_idf_dict in tf_idf.get_tf_idf_dict().items():
    print doc 
    print tf_idf_dict